In [2]:
import pandas as pd
import re

email_data = pd.read_csv('C:/Users/User/OneDrive - Asia Pacific University/APU Final Year Project/FYP Email Project Documents/Email Dataset Python/FYP PART 1/enron_cleaned_new_feature.csv')

In [3]:
email_data.info(memory_usage='deep') 
print("Number of Rows and Columns: ", email_data.shape) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253175 entries, 0 to 253174
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   From           253175 non-null  object
 1   To             243285 non-null  object
 2   Subject        244469 non-null  object
 3   Message        253174 non-null  object
 4   Cc             59495 non-null   object
 5   Bcc            59495 non-null   object
 6   Day            253175 non-null  object
 7   Date & Time    253175 non-null  object
 8   Response Time  253175 non-null  object
dtypes: object(9)
memory usage: 561.8 MB
Number of Rows and Columns:  (253175, 9)


In [5]:
forward_pattern = r"----- Forwarded by (.+?) on (\d{2}/\d{2}/\d{4} \d{2}:\d{2} [APM]{2}) -----"
from_pattern = r"From: (.+?)"
to_pattern = r"To: (.+?)(?:\n|cc:)"
subject_pattern = r"Subject: (.*?)\n"
body_pattern = r"(?:Subject: .*?\n)(.*?)(?=----- Forwarded by|$)"

def extract_metadata(email):
    if not isinstance(email, str):
        return None

    all_metadata = []

    forwarding_blocks = re.findall(forward_pattern, email)
    for forward_by, forward_on in forwarding_blocks:
        metadata = {"Forwarded by": forward_by.strip(), "On": forward_on.strip()}

        block_match = re.search(
            f"----- Forwarded by {re.escape(forward_by)} on {re.escape(forward_on)} -----(.*?)(?:----- Forwarded by|$)",
            email,
            flags=re.DOTALL,
        )
        block_content = block_match.group(1).strip() if block_match else ""

        from_match = re.search(from_pattern, block_content)
        to_match = re.search(to_pattern, block_content)
        subject_match = re.search(subject_pattern, block_content)
        body_match = re.search(body_pattern, block_content, flags=re.DOTALL)

        metadata["From"] = from_match.group(1).strip() if from_match else None
        metadata["To"] = to_match.group(1).strip() if to_match else None
        metadata["Subject"] = subject_match.group(1).strip() if subject_match else None
        metadata["Body"] = body_match.group(1).strip() if body_match else None

        all_metadata.append(metadata)

    if not all_metadata:
        metadata = {}
        from_match = re.search(from_pattern, email, flags=re.MULTILINE)
        to_match = re.search(to_pattern, email, flags=re.MULTILINE)
        subject_match = re.search(subject_pattern, email, flags=re.MULTILINE)
        body_match = re.search(body_pattern, email, flags=re.DOTALL)

        metadata["From"] = from_match.group(1).strip() if from_match else None
        metadata["To"] = to_match.group(1).strip() if to_match else None
        metadata["Subject"] = subject_match.group(1).strip() if subject_match else None
        metadata["Body"] = body_match.group(1).strip() if body_match else None

        all_metadata.append(metadata)

    return all_metadata

email_data['Extracted Metadata'] = email_data['Message'].apply(extract_metadata)


In [ ]:
email_data

In [12]:
def fix_split_words_with_spellchecker(message):
    if not isinstance(message, str):
        return message  # If it's NaN (float), just return it as-is (or return an empty string if preferred)

    words = message.split()
    fixed_words = []
    i = 0

    while i < len(words):
        if i < len(words) - 1:
            combined = words[i] + words[i+1]
            if combined.lower() in spell:
                fixed_words.append(combined)
                i += 2
            else:
                fixed_words.append(words[i])
                i += 1
        else:
            fixed_words.append(words[i])
            i += 1

    return ' '.join(fixed_words)

# Fill NaN values (recommended for stable processing)
email_data['Message'] = email_data['Message'].fillna('')
email_data['Subject'] = email_data['Subject'].fillna('')

# Apply the function
email_data['Message'] = email_data['Message'].apply(fix_split_words_with_spellchecker)
email_data['Subject'] = email_data['Subject'].apply(fix_split_words_with_spellchecker)

In [13]:
email_data

,From,To,Subject,Message,Cc,Bcc,Day,Date & Time,Response Time,Extracted Metadata,url_count,email_address_count,attachment_count
0,frank.billington@oatiinc.com,cara.semperger@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:25:00,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
1,frank.billington@oatiinc.com,jeff.king@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:28:51,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
2,frank.billington@oatiinc.com,phillip.platter@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:32:28,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
3,frank.billington@oatiinc.com,diana.scholtes@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 23:26:22,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
4,david.steiner@enron.com,center.dl-portland@enron.com,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,Immediately delete and DO NOT OPEN email From:...,NaN,NaN,Tuesday,2001-12-04 19:49:46,NaT,"[{'From': 'A', 'To': None, 'Subject': None, 'B...",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253170,john.ruiz@enron.com,"jeff.duff@enron.com, mark.walker@enron.com, ma...",,The Weekly Availabilty Status report and excel...,NaN,NaN,Wednesday,2002-07-10 07:48:00,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0
253171,cgehring@renewdata.com,NaN,,--------- Inline attachment follows --------- ...,NaN,NaN,Thursday,2002-09-19 00:34:42,NaT,"[{'From': '=', 'To': None, 'Subject': None, 'B...",0,0,0
253172,cgehring@renewdata.com,NaN,,"------------ Sent: Monday, August 27, 2001 1:1...",NaN,NaN,Thursday,2002-09-19 00:35:11,NaT,"[{'From': 'D', 'To': None, 'Subject': None, 'B...",0,0,0
253173,cgehring@renewdata.com,NaN,,--------- Inline attachment follows --------- ...,NaN,NaN,Thursday,2002-09-19 19:59:09,NaT,"[{'From': 'T', 'To': None, 'Subject': None, 'B...",0,0,0


In [7]:
import re

def extract_counts(email_text):
    if not isinstance(email_text, str):
        return 0, 0, 0  # Handle missing or non-string values safely

    # Count URLs
    url_count = len(re.findall(r"http[s]?://\S+", email_text))

    # Count Email Addresses
    email_address_count = len(re.findall(r"\b\S+@\S+\b", email_text))

    # Count Attachments
    attachment_count = len(re.findall(r"<< File:.*?>>", email_text))

    return url_count, email_address_count, attachment_count

email_data[['url_count', 'email_address_count', 'attachment_count']] = email_data['Message'].apply(
    lambda x: pd.Series(extract_counts(x))
)


In [8]:
email_data

,From,To,Subject,Message,Cc,Bcc,Day,Date & Time,Response Time,Extracted Metadata,url_count,email_address_count,attachment_count
0,frank.billington@oatiinc.com,cara.semperger@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:25:00,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
1,frank.billington@oatiinc.com,jeff.king@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:28:51,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
2,frank.billington@oatiinc.com,phillip.platter@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 19:32:28,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
3,frank.billington@oatiinc.com,diana.scholtes@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,NaN,NaN,Friday,2002-02-01 23:26:22,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0
4,david.steiner@enron.com,center.dl-portland@enron.com,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,Immediately delete and DO NOT OPEN email From:...,NaN,NaN,Tuesday,2001-12-04 19:49:46,NaT,"[{'From': 'A', 'To': None, 'Subject': None, 'B...",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253170,john.ruiz@enron.com,"jeff.duff@enron.com, mark.walker@enron.com, ma...",NaN,The Weekly Availabilty Status report and excel...,NaN,NaN,Wednesday,2002-07-10 07:48:00,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0
253171,cgehring@renewdata.com,NaN,NaN,--------- Inline attachment follows --------- ...,NaN,NaN,Thursday,2002-09-19 00:34:42,NaT,"[{'From': '=', 'To': None, 'Subject': None, 'B...",0,0,0
253172,cgehring@renewdata.com,NaN,NaN,"------------ Sent: Monday, August 27, 2001 1:1...",NaN,NaN,Thursday,2002-09-19 00:35:11,NaT,"[{'From': 'D', 'To': None, 'Subject': None, 'B...",0,0,0
253173,cgehring@renewdata.com,NaN,NaN,--------- Inline attachment follows --------- ...,NaN,NaN,Thursday,2002-09-19 19:59:09,NaT,"[{'From': 'T', 'To': None, 'Subject': None, 'B...",0,0,0


In [14]:
import re
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])  # Only tokenizer + lemmatizer

def advanced_cleaning_pipeline(email_text):
    if not isinstance(email_text, str):
        return ""

    # 1. Remove forwarded/original headers (multi-line removal)
    email_text = re.sub(r"(?is)-----.*?Subject:", "", email_text)

    # 2. Normalize all newlines, tabs, excessive spaces
    email_text = re.sub(r"[\r\n\t]+", " ", email_text)
    email_text = re.sub(r"\s+", " ", email_text).strip()

    # 3. Fix common email encoding artifacts
    email_text = re.sub(r"=20", " ", email_text)  # Quoted printable space
    email_text = re.sub(r"=09", " ", email_text)  # Quoted printable tab
    email_text = re.sub(r"=3D", "=", email_text)  # Quoted printable equals
    email_text = re.sub(r"=\s?", "", email_text)  # Quoted printable soft line break

    # 4. Remove URLs, Emails, Attachments (no placeholders - full removal)
    email_text = re.sub(r"http[s]?://\S+", "", email_text)
    email_text = re.sub(r"\b\S+@\S+\b", "", email_text)
    email_text = re.sub(r"<< File:.*?>>", "", email_text)

    # 5. Remove common signatures/closings
    email_text = re.sub(r"(?i)(thanks|regards|sincerely|best),?", "", email_text)

    # 6. Optional: Remove phone numbers
    email_text = re.sub(r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}", "", email_text)

    # 7. Final cleanup - extra spaces from removals
    email_text = re.sub(r"\s+", " ", email_text).strip()

    # 8. Run spaCy NLP (tokenization, lemmatization, stopword removal)
    doc = nlp(email_text)

    # 9. Keep only lemmatized tokens (no stopwords, no punctuation, no numbers)
    cleaned_tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct and not token.like_num
    ]

    cleaned_text = " ".join(cleaned_tokens)

    # 10. Optional: Truncate to max 300 words (recommended for classification)
    max_length = 300
    if len(cleaned_text.split()) > max_length:
        cleaned_text = " ".join(cleaned_text.split()[:max_length])

    return cleaned_text

email_data['Cleaned_Message'] = email_data['Message'].apply(advanced_cleaning_pipeline)

Index(['From', 'To', 'Subject', 'Message', 'Day', 'Date & Time',
       'Response Time', 'Extracted Metadata', 'url_count',
       'email_address_count', 'attachment_count', 'Cleaned_Message'],
      dtype='object')

In [ ]:
email_data = email_data.drop(columns=['Cc', 'Bcc'])

In [106]:
import pandas as pd
import re

email_data = pd.read_csv('C:/Users/User/OneDrive - Asia Pacific University/APU Final Year Project/FYP Email Project Documents/Email Dataset Python/FYP PART 1/email_data_cleaning_halfway.csv')

In [107]:
def universal_artifact_cleaner(text):
    if not isinstance(text, str):
        return ""

    # Common encoding artifacts (Quoted Printable)
    text = re.sub(r'=\s?', '', text)   # Soft line breaks
    text = re.sub(r'=20', ' ', text)   # Space encoding
    text = re.sub(r'=09', ' ', text)   # Tab encoding
    text = re.sub(r'=3D', '=', text)   # Equals sign encoding

    # Weird numerical artifacts (like 01,s for 's)
    text = re.sub(r'(\d{2})[,\.](\w)', r'\2', text)  # Remove numeric prefix
    text = re.sub(r'\b(\d{2})\b', '', text)          # Remove stray numbers if isolated (optional)

    # Fix apostrophe handling
    text = re.sub(r'01,s', "'s", text)
    text = re.sub(r'01,t', "'t", text)

    # Common email artifacts (forwarded/reply headers)
    text = re.sub(r'(?is)-----.*?Subject:', '', text)  # Forwarded/Original block

    # Remove weird standalone letters (like v)
    text = re.sub(r'\bv\b', '', text)

    # Extra space cleanup
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(universal_artifact_cleaner)

In [108]:
import re

def further_clean_process_guidelines(text):
    if not isinstance(text, str):
        return ""

    # Remove step numbering like "1a", "2b", "3b"
    text = re.sub(r'\b\d+[a-zA-Z]?\b', '', text)

    # Normalize excessive spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Optional: Capitalize important entities (depends on your goal)
    entities = ['oati', 'nerc', 'frank billington', 'etag']
    for entity in entities:
        text = re.sub(rf'\b{entity}\b', entity.upper(), text)

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(further_clean_process_guidelines)

In [109]:
def normalize_abbreviations(text):
    abbr_mapping = {
        "u.s.": "us",
        "u.s.-based": "us-based",
        "e.u.": "eu",
        "u.k.": "uk",
        "a.s.a.p.": "asap",
        "c.e.o.": "ceo",
        "c.f.o.": "cfo",
        "e.g.": "for example",
        "i.e.": "that is"
    }
    for key, value in abbr_mapping.items():
        text = text.replace(key, value)
    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(normalize_abbreviations)

In [110]:
def advanced_post_clean(text):
    if not isinstance(text, str):
        return ""

    # Remove internal artifacts like extension numbers and placeholders
    text = re.sub(r'x\d{4,6}', '', text)

    # Remove email domains
    text = re.sub(r'\b\S+@\S+\b', '', text)

    # Fix dollar amounts (optional - domain dependent)
    text = re.sub(r'\$', 'dollar ', text)

    # Restore some contractions (optional)
    text = re.sub(r"wont", "won't", text)

    # Add rough sentence breaks based on detected topic switches
    text = re.sub(r"(enron|spearman|library|fundraising|notice|match)", r". \1", text)

    # Normalize excessive spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(advanced_post_clean)

In [111]:
import re

def cleanup_announcement_email(text):
    if not isinstance(text, str):
        return ""

    # Remove scheduling blocks (times, rotators, broadcasts)
    text = re.sub(r'(\d{1,2}(?::\d{2})?(am|pm|AM|PM)[\s\S]*?eyewitness news)', '', text)
    text = re.sub(r'(\d{1,2}(?::\d{2})?(am|pm|AM|PM))', '', text)

    # Remove person contact detail (name + extension if present)
    text = re.sub(r'laura schwartz', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\breach x\d+\b', '', text)

    # Remove "bb" and other internal scheduling junk
    text = re.sub(r'\bbb\b', '', text)

    # Remove redundant repeated phrases (like "eyewitness news")
    text = re.sub(r'\beyewitness news\b', '', text, flags=re.IGNORECASE)

    # Normalize spaces after removals
    text = re.sub(r'\s+', ' ', text).strip()

    # Optional - restore some sentence segmentation based on structure cues
    text = re.sub(r'(?i)\benron\b', '. Enron', text)
    text = re.sub(r'(?i)\bsponsor\b', '. Sponsor', text)
    text = re.sub(r'(?i)\bspecial\b', '. Special', text)

    return text.lower()  # or remove .lower() if using BERT

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(cleanup_announcement_email)

In [112]:
import re

def further_process_flat(text):
    if not isinstance(text, str):
        return ""

    # Remove repeated periods or replace with space (you want no noise from this)
    text = re.sub(r'\s*\.\s*\.\s*', ' ', text)

    # Optional: Remove ALL periods if you want zero punctuation
    text = re.sub(r'\.', ' ', text)

    # Normalize spacing
    text = re.sub(r'\s+', ' ', text).strip()

    # Lowercase everything (best practice for TF-IDF)
    text = text.lower()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(further_process_flat)

In [113]:
import re

def clean_broadcast_announcement(text):
    if not isinstance(text, str):
        return ""

    # Remove times and slashes
    text = re.sub(r'[:]', ' ', text)
    text = re.sub(r'//', ' ', text)

    # Remove scheduling mentions
    text = re.sub(r'\b(moneyscope|nightline|report)\b', '', text)

    # Remove "reach x" type placeholders
    text = re.sub(r'\breach x\d+\b', '', text)

    # Fix p m to pm
    text = re.sub(r'\bp m\b', 'pm', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Lowercase for TF-IDF consistency
    text = text.lower()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_broadcast_announcement)

In [114]:
import re

def fix_email_split(text):
    if not isinstance(text, str):
        return ""

    # Normalize all variations to "email"
    text = re.sub(r'\b(e[\s\-.]?mail)\b', 'email', text, flags=re.IGNORECASE)

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(fix_email_split)

In [115]:
import re

def clean_promo_email_symbol_free(text):
    if not isinstance(text, str):
        return ""

    # Fix encoding artifacts and remove problematic symbols
    text = text.replace("men?s", "mens")
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)  # Remove all symbols

    # Remove pricing and sizes entirely (not needed for classification)
    text = re.sub(r'price dollar|reg dollar|save dollar', '', text)
    text = re.sub(r'm l|l xl', '', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Lowercase everything for consistent TF-IDF input
    return text.lower()

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_promo_email_symbol_free)

In [116]:
import re

def fix_h_s_trask_issue(text):
    if not isinstance(text, str):
        return ""

    # Merge known brand references (custom rule for your data)
    text = text.replace("h s trask", "hstrask")

    # Optionally remove all isolated single letters (if you want to clean up general noise)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(fix_h_s_trask_issue)

In [117]:
#Helps models learn correct associations (G7 relates to finance, clients, surveys, etc.).
import re

def enhance_g7(text):
    if not isinstance(text, str):
        return ""

    # Replace isolated g7 with "g7 group"
    text = re.sub(r'\bg7\b', 'g7 group', text)

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(enhance_g7)

In [118]:
import re

def replace_userid(text):
    # Replace patterns like "campb54792" with a generic token
    text = re.sub(r'\buserid campb54792b\d+\b', 'userid', text)
    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(replace_userid)

In [119]:
import re

def clean_final_message(text):
    if not isinstance(text, str):
        return ""

    # Remove repeated file mentions
    words = text.split()
    deduplicated_words = []
    seen_words = set()
    
    for word in words:
        if word not in seen_words:
            deduplicated_words.append(word)
            seen_words.add(word)

    cleaned_text = ' '.join(deduplicated_words)

    # Normalize attachment mention to simple "attachment"
    cleaned_text = re.sub(r'\battached file\b', 'attachment', cleaned_text)

    # Optional: Remove filename-like tokens (if filenames don't matter for classification)
    cleaned_text = re.sub(r'\bericinvite doc\b', 'attachment', cleaned_text)

    # Final space cleanup
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_final_message)

In [120]:
import re

def normalize_attachment_mention(text):
    if not isinstance(text, str):
        return ""

    # Replace filename patterns like "mvc jpg" or "ericinvite doc" with generic "attachment"
    text = re.sub(r'\bmvc jpg\b', 'attachment image', text)

    # Optional — If you want ALL file types to collapse to just "attachment"
    # text = re.sub(r'\b\w+\s(?:jpg|png|doc|pdf|xls)\b', 'attachment', text)

    # Normalize spacing
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(normalize_attachment_mention)

In [121]:
import re

def clean_promo_text(text):
    if not isinstance(text, str):
        return ""

    # Fix common typos
    text = text.replace("comission", "commission")

    # Optional — Standardize some promo terms (depends on your goal)
    text = text.replace("costless cup coffee day", "low cost daily")

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_promo_text)


In [122]:
import re

def normalize_attachment_filename(text):
    if not isinstance(text, str):
        return ""

    # Case 1: Convert to "attachment spreadsheet"
    text = re.sub(r'\bmarket briefs xls\b', 'attachment spreadsheet', text)

    # Optional Case 2: Convert to "attachment market briefs" if filename matters
    # text = re.sub(r'\bmarket briefs xls\b', 'attachment market briefs', text)

    # Final space cleanup
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(normalize_attachment_filename)


In [123]:
import re

def clean_html_email(text):
    if not isinstance(text, str):
        return ""

    # Remove HTML tags
    text = re.sub(r'<[^>]*>', ' ', text)

    # Remove doctype, metadata, and formatting junk
    text = re.sub(r'doctype html public.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'alink\s+\w+|link\s+\w+|vlink\s+\w+', '', text)
    text = re.sub(r'(table|tr|td|colspan|cellpadding|cellspacing|width|height|bgcolor|align|border|usemap|script|language|noscript|map|coords)', '', text)

    # Replace HTML entities
    text = text.replace('&nbsp;', ' ')

    # Remove excessive spaces and leftover artifacts
    text = re.sub(r'\s+', ' ', text).strip()

    # Lowercase if desired
    text = text.lower()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(normalize_attachment_filename)

In [124]:
import re

html_like_terms = {
    "doctype", "html", "public", "w3c", "dtd", "transitional", "head", "body",
    "alink", "link", "vlink", "bgcolor", "center", "table", "tr", "td", "src",
    "width", "height", "cellpadding", "cellspacing", "bgcolor", "font", "face",
    "size", "color", "script", "noscript", "menu", "map", "area", "coords",
    "colspan", "alt", "border", "usemap"
}

def clean_html_text_without_tags(text):
    if not isinstance(text, str):
        return ""

    # Remove known HTML terms
    words = text.split()
    cleaned_words = [word for word in words if word.lower() not in html_like_terms]
    text = ' '.join(cleaned_words)

    # Remove styling junk (width716, bgcolorblack)
    text = re.sub(r'\b(?:width|height|border|bgcolor|face|size|color|alink|vlink|link)\w*\b', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()  # if needed

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_html_text_without_tags)


In [125]:
import re

def replace_identifiers(text):
    if not isinstance(text, str):
        return ""

    # General patterns for different identifier types
    text = re.sub(r'\bopl\d+\b', 'block_id', text)
    text = re.sub(r'\bid\d+\b', 'deal_id', text)
    text = re.sub(r'\binv\d+\b', 'invoice_id', text)
    text = re.sub(r'\bcontract\d+\b', 'contract_id', text)
    text = re.sub(r'\bfile\d+\b', 'file_id', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(replace_identifiers)

In [126]:
import re

def remove_html_artifacts(text):
    if not isinstance(text, str):
        return ""

    # Remove color codes (hex like ffffff or ffcc00)
    text = re.sub(r'\b[0-9a-fA-F]{6}\b', '', text)

    # Remove common HTML table/layout formatting terms
    html_junk = [
        'cellpadding', 'cellspacing', 'valign', 'align',
        'gutter', 'block', 'header', 'banner', 'logo', 'openlink'
    ]
    for junk in html_junk:
        text = re.sub(rf'\b{junk}\w*\b', '', text)

    # Remove font-family leftovers (like arial helvetica sans serif)
    text = re.sub(r'arial|helvetica|sans serif', '', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()  # optional, if your pipeline wants lowercase

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(remove_html_artifacts)

In [127]:
import re

def clean_travel_email(text):
    if not isinstance(text, str):
        return ""

    # Remove all ippageid artifacts (tracking codes)
    text = re.sub(r'\bippageid\d+\b', '', text)

    # Fix flight route patterns (portland united states pdx intl dallas tx dfw)
    text = re.sub(r'\b(pdx|dfw|ord|okc|lax|jfk|sfo|iah|den|atl)\b', lambda m: m.group(0).upper(), text)

    # Optional: Convert "intl" to "international" for better clarity
    text = text.replace('intl', 'international')

    # Normalize spacing
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()  # Optional depending on your feature strategy

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_travel_email)

In [128]:
import re

def fix_split_words(text):
    if not isinstance(text, str):
        return ""

    # Common split fixes
    split_fixes = {
        "ser vice": "service",
        "inter national": "international",
        "in dustry": "industry",
        "tele phony": "telephony",
        "sub continent": "subcontinent",
        "viet natd": "vietnam"
    }

    for broken, fixed in split_fixes.items():
        text = text.replace(broken, fixed)

    # Clean up spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(fix_split_words)

In [129]:
import re

def clean_broken_urls(text):
    if not isinstance(text, str):
        return ""

    # Case 1: Remove all URLs completely (cleanest)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Optional Case 2: Replace URLs with "url" (keeps URL presence signal)
    # text = re.sub(r'http\S+|www\S+', 'url', text)

    # Optional Case 3: If baker mckenzie should be retained
    text = text.replace('www bakernet com', 'baker mckenzie')

    # Clean extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_broken_urls)

In [130]:
import re

def detect_html_email(message):
    if not isinstance(message, str):
        return False

    # These are common signs of HTML-heavy emails
    patterns = [
        r'<html', r'<body', r'<table', r'<img', r'<a href', r'<span', 
        r'<link', r'<title', r'<head', r'<style'
    ]
    
    # If any of these appear in the message, it's likely an HTML-heavy email
    return any(re.search(p, message, re.IGNORECASE) for p in patterns)

# Create a new column flagging HTML-heavy emails
email_data['Is_HTML_Heavy'] = email_data['Message'].apply(detect_html_email)

# Count how many emails have this pattern
html_heavy_count = email_data['Is_HTML_Heavy'].sum()

print(f"Number of HTML-heavy emails: {html_heavy_count}")

Number of HTML-heavy emails: 2109


In [131]:
# Remove HTML-heavy emails directly
email_data = email_data[~email_data['Is_HTML_Heavy']]

# Drop the helper column
email_data = email_data.drop(columns=['Is_HTML_Heavy'])

In [132]:
import re

def remove_urls(email_text):
    if not isinstance(email_text, str):
        return ""  # Return empty string if the input is not a string
    
    # Remove URL-like patterns (e.g., "http www expedia com", "http://www.example.com", "www.example.com")
    email_text = re.sub(r'\b(?:http|https|www)\S*\b', '', email_text)

    # Remove excessive spaces
    email_text = re.sub(r'\s+', ' ', email_text).strip()

    return email_text

# Apply to your DataFrame
email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(remove_urls)


In [181]:
import re

def clean_text(text):
    text = re.sub(r'(?i)(scheduling)(iso)', r'\1 iso', text)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

    corrections = {
        "portlandwestdeskcalifornia": "portland west desk california",
        "schedulestxt": "schedules txt",
        "hourahead": "hour ahead",
        "enroncc": "enron cc",
        "imageimage": "image",
        "amt": "amount",
        "ie": "example",
        "mth": "month",
        "gonescr": "gone scr",
    }

    for wrong, correct in corrections.items():
        text = re.sub(r'\b' + re.escape(wrong) + r'\b', correct, text, flags=re.IGNORECASE)

    remove_words = r'\b(aa|ab|eb|im|txu|iso|txt|ect|hou|abb|ad|abx|ac|abb|abel|abc|aei|aeps|ieee|apsepmiciso|badlndplx|calgenunit|caps|da ha da hamp|ect)\b'
    text = re.sub(remove_words, '', text, flags=re.IGNORECASE).strip()

    return text

email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_text)

C:\Users\User\AppData\Local\Temp\ipykernel_6592\4018295038.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_data['Cleaned_Message'] = email_data['Cleaned_Message'].apply(clean_text)


In [182]:
email_data

,From,To,Subject,Message,Day,Date & Time,Response Time,Extracted Metadata,url_count,email_address_count,attachment_count,Cleaned_Message,Cleaned_Subject,Combined_Text
0,frank.billington@oatiinc.com,cara.semperger@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,Friday,2002-02-01 19:25:00,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0,time run short company prepare etag minimum re...,oati etag minimum requirement,oati etag minimum requirement time run short c...
1,frank.billington@oatiinc.com,jeff.king@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,Friday,2002-02-01 19:28:51,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0,time run short company prepare etag minimum re...,oati etag minimum requirement,oati etag minimum requirement time run short c...
2,frank.billington@oatiinc.com,phillip.platter@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,Friday,2002-02-01 19:32:28,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0,time run short company prepare etag minimum re...,oati etag minimum requirement,oati etag minimum requirement time run short c...
3,frank.billington@oatiinc.com,diana.scholtes@enron.com,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,Friday,2002-02-01 23:26:22,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",4,0,0,time run short company prepare etag minimum re...,oati etag minimum requirement,oati etag minimum requirement time run short c...
4,david.steiner@enron.com,center.dl-portland@enron.com,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,Immediately delete and DO NOT OPEN email From:...,Tuesday,2001-12-04 19:49:46,NaT,"[{'From': 'A', 'To': None, 'Subject': None, 'B...",0,0,0,immediately delete open email alam raf subject...,gonescr virus warning,gonescr virus warning immediately delete open ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253165,joe.parks@enron.com,stanley.k.horton@dynegy.com,NaN,"both list are on here, the one you are looking...",Wednesday,2002-06-19 17:05:38,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0,list look,,list look
253166,chris.germany@enron.com,chris.germany@enron.com,NaN,Date Daily Withdrawal Daily Injection Jul-02 3...,Wednesday,2002-06-19 21:49:43,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0,date daily withdrawal injection jul aug sep oc...,,date daily withdrawal injection jul aug sep oc...
253167,chris.germany@enron.com,chris_germany61@hotmail.com,NaN,good luck. when is comes to daycares. it proba...,Thursday,2002-06-20 16:15:59,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0,good luck come daycare probably look sense pla...,,good luck come daycare probably look sense pla...
253168,lisa.gang@enron.com,eemainzer@bpa.gov,NaN,hey sweet cheeks....how the hell are ya????how...,Friday,2002-06-21 15:37:02,NaT,"[{'From': None, 'To': None, 'Subject': None, '...",0,0,0,hey sweet cheek hell ya how bpa bet let know v...,,hey sweet cheek hell ya how bpa bet let know v...


# Subject Cleaning

In [133]:
email_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251066 entries, 0 to 253174
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   From                 251066 non-null  object
 1   To                   241206 non-null  object
 2   Subject              242373 non-null  object
 3   Message              251065 non-null  object
 4   Day                  251066 non-null  object
 5   Date & Time          251066 non-null  object
 6   Response Time        251066 non-null  object
 7   Extracted Metadata   251065 non-null  object
 8   url_count            251066 non-null  int64 
 9   email_address_count  251066 non-null  int64 
 10  attachment_count     251066 non-null  int64 
 11  Cleaned_Message      251066 non-null  object
dtypes: int64(3), object(9)
memory usage: 24.9+ MB


In [134]:
import re

# Define list of file extensions to catch file-only emails
file_extensions = ['.xls', '.xlsx', '.pdf', '.doc', '.docx', '.ppt', '.pptx', '.jpg', '.png', '.zip']

def is_valid_email(row):
    subject = str(row['Subject']).strip().lower()
    message = str(row['Message']).strip().lower()

    # 1️⃣ Subject is nan and message is just a link
    if subject == 'nan' and re.match(r'^https?://\S+$', message):
        return False

    # 2️⃣ Subject is nan and message starts with attachment-like content
    if subject == 'nan' and message.startswith('-'):
        return False

    # 3️⃣ Extremely short "nan" emails (fewer than 5 words)
    if subject == 'nan' and len(message.split()) < 5:
        return False

    # 4️⃣ Subject is a file name and message repeats the same file name (file-only email)
    for ext in file_extensions:
        if subject.endswith(ext) and message.startswith(f'- {subject}'):
            return False

    # 5️⃣ Message contains ONLY a file name (e.g., "- file.xls")
    if re.match(r'^-\s*\S+\.(xls|xlsx|pdf|doc|docx|ppt|pptx|jpg|png|zip)$', message):
        return False

    # 6️⃣ Message contains ONLY a link
    if re.match(r'^https?://\S+$', message):
        return False

    return True

# ✅ Apply this filter to your DataFrame
email_data = email_data[email_data.apply(is_valid_email, axis=1)]

print(f"✅ After removing junk emails, {len(email_data)} rows remain.")

✅ After removing junk emails, 249051 rows remain.


In [135]:
# Define the specific subject-message pairs to remove
emails_to_remove = [
    ("Barnaby's", "http://houston.citysearch.com/E/V/HOUTX/0000/91/67/cs1.html"),
    ("Altrade", "https://mesa.altra.com/production/index.html"),
    ("hi", "o zemra ime e vogel, te dua shume. Me ka marre malli mua per ty"),
    ("reuters stroy", "check this out"),
    ("An Ancient Demon?", "If you dare. http://www.mrwinkle.com/"),
    ("Caution ... bad language", "- dab.exe"),
    ("nan", "call me!!!! i am so bored!"),
    ("Re:", "Perhaps..."),
    ("nan", "olandis gary tore his acl. he is out for the season."),
    ("Morning Download is Complete", "Robert B Cothran Enron Americas 713-853-3478"),
    ("nan", "Dad, Canadian 88 up $0.25 to $2.05"),
    ("RE:", "huh ?"),
    ("SCS CRUDE STRADDLES 11-27", "- 11_27_01LO.xls"),
    ("11-20", "- nov2001.XLS"),
    ("Re: EnronOnline-Entergy", "Ughh..."),
    ("for your enjoyment", "<http://slate.msn.com/?id=2061505>"),
    ("RE:", "No."),
    ("nan", "My 1,4,and 6 for your 1, 5,7"),
    ("RE:", "http://www.link.duke-energy.com/"),
    ("RE:", "what is lz's?"),
    ("Re:", "30-40"),
    ("nan", "you bot 475 at 11.75"),
    ("ITS A PACIFIER.", "? - TEEF.JPG"),
    ("Caution ... bad language", "- dab.exe"),
    ("Ouch for Texas", "http://sportsillustrated.cnn.com/football/college/news/2001/05/27/offseason_ beat/"),
    ("Re: logins", "EOL Logon: emclaug errol5enter"),
    ("Fw: Dems Make New Ballot for Florida Elderly", "? [IMAGE] - elderly ballot.gif"),
    ("USBS Painewebber", "hrgis#: 90010251 pin#: 9257"),
    ("Fw: Mini Me", "? - MiniBush.jpg"),
    ("agraeg:", "afwsfg"),
    ("quotesen", "<<quotesen.xls>> quotesen"),
    ("FW: Palm Beach Precinct", "more :) - Far_side.jpg"),
    ("http://users.aimnet.com/~ksyrah/ekskurs/lit.html", "http://studyrussian.com/literature/main.html http://users.aimnet.com/~ksyrah/ekskurs/lit.html"),
    ("Re: suicide pool - nfl 2000", "i'm in"),
    ("5 X 24", "Any news? Eric"),
    ("Oops!", "- w&N)TS.mpe"),
    ("Sleepy", "? - Just my luck.gif"),
    ("FW: Intelligence Test", "- intelligentietest.htm"),
    ("Fw: another ballot", "? - ballot[1].gif"),
    ("goo", "713-334-6106 http://www.oilonline.com/equip/serv_pro.asp"),
    ("tdwaterhouse_2000", "gif"),
    ("tdwaterhouse_2000", "tdwaterhouse_2000"),
    ("FW: Yellow pages", "Bad Day Yellowpages.mpg"),
    ("Re:", "at 5:00"),
    ("range", "Carol"),
    ("Kristin Stanton", "Western Division"),
    ("Re:", "Thanks"),
    ("Re: Klauberg", "I will"),
    ("New Rates #7A63", "***** This is an HTML Message ! *****"),
    ("Re:", "ditto"),
    ("Fw: Santa", "? - xmassuit.gif"),
    ("Brenda - Vacation", "__________________ __________________"),
    ("#2", "- MVC-010S.JPG - MVC-007S.JPG - MVC-008S.JPG - MVC-009S.JPG - MVC-006S.JPG"),
    ("esgsea", "eagaerg"),
    ("Re:", "Done"),
    ("Re: Rebooks", "Done"),
    ("nan", "My 1,4,and 6 for your 1, 5,7"),
    ("Re:", "rite spot. 17th and folsom. the best. Best, Jeff"),
    ("Brenda - Vacation", "__________________"),
    ("Re:", "no"),
    ("asdf", "asdfasd"),
    ("Re:", "mackle's # is 781-639-2752"),
    ("http://www.nibblebox.com/public/nibbles/bigbunny/big_bunny_winfre y.shtml", "?"),
    ("Re:", "Dwivedi"),
    ("Re: Re[2]:", "who is inga?"),
    ("Re:", "713-853-6203"),
    ("RE: where are you", "http://howto.yahoo.com/ask/19990111a.html J"),
    ("Re:", "i just got a 28"),
    ("Re:", "Why"),
    ("RE: tolling", "Thanks!"),
    ("Re:", "allegedly"),
    ("RE:", "call me 713-853-0685"),
    ("here you go", "? - wheellg.bmp - wof1.bmp - wof2.bmp"),
    ("Re: Badge Nos.", "26510"),
    ("smp", "smp @ 800"),
    ("Re:", "1996. it had 65k. $18k."),
    ("Re:", "Message: i got an 86."),
    ("Re:", "Thanks!"),
    ("To_do", "1. M. Soto"),
    ("?", "sniff sniff"),
    ("RE:", "xoxoxoxoxoxoxoxoxoxox"),
    ("Re: Badge Nos.", "badge no 11592"),
    ("WASSUP!!!!!!", "- Wassup.avi"),
    ("Re: Seaboard", "Nope..."),
    ("RE:", "21.89"),
    ("", "tb + 9.5 for 250"),
    ("CRRA LOI", "no"),
    ("Re", "Enjoy!"),
    ("Re:", "37176"),
    ("Re: Re: ..", "no"),
    ("Re", "646-3393"),
    ("CMB JPM", "Ouch!"),
    ("RE:", "So........."),
    ("nan", "you bot 475 at 11.75"),
    ("Re:", "mediatrip.com"),
    ("Re:", "Weasel!!"),
    ("Re:", "11"),
    ("Re: Re[2]: oath to you...", "sailing"),
    ("Re: Votoratim", "Either"),
    ("HRGIS#90013019", "90013019"),
    ("nan", "My 1,4,and 6 for your 1, 5,7"),
    ("call me you freak x39837", "CKL"),
    ("http://www.sharperimage.com/us/en/catalog/productview.jhtml?pid=15975000&pcatid=22&catid=22", "http://www.sharperimage.com/us/en/catalog/productview.jhtml?pid=15975000&pcati d=22&catid=22"),
    ("Call Me", "853-5926"),
    ("nan", 'W:SCRIPTSexceed.vbs "h:exceed62userrunm.xs" Truong Vu x 5-3416'),
    ("Re:", "$65"),
    ("palmbeachvoter", "? - palmbeachvoter.jpg"),
    ("Re:", "Thanks!!!"),
    ("Hawker bday", "Byob"),
    ("Re:", "$65"),
    ('Rovner "Shayara"', "- Rovner Shayara III.jpg"),
    ("CRRA LOI", "Enjoy!"),
    ("Call Me", "853-5926"),
    ("me", "me and my friend julia"),
    ("CED", "Call me. 713.853.9351"),
    ("Re:", "Wed 12:30 ?"),
    ("Re: Howdy", "Saw"),
    ("Re: Txt txt tx-T-T-T Txt txt tx-TTT T-X-T!!!! Tx-t-x-t ! (sung to the tune of 'The Lone Ranger')", "Now I'm scared."),
    ("Ormet", "Here it is!"),
    ("Re:", "soy is discusting"),
    ("TXN", "Up 8.5%!!!!, for now"),
    ("fundies", "call me"),
    ("well????", "I WANT DETAILS!!!!"),
    ("Fuzzy Math!", "http://www.lasvegassun.com/sunbin/stories/archives/2000/nov/10/511018638.html? tom+carroll"),
    ("me", "me and my grandma"),
    ("RE: thanks", "no"),
    ("Re:", "queen"),
    ("yahoooooooooooooooooooo", "i've lost my link to NYMEXINFO.COM"),
    ("t", "lasjkdfakl; sdal;sdj ;alsdfj kl;asdjf"),
    ("Cheaties - Low Chad - Political humor", "? ? ? ????????[IMAGE] - windowsTEMPnsmailAE.jpeg"),
    ("", "giants -7 giants - dallas minus 37 both 400"),
    ("Harriet Turk", "901-351-5706"),
    ("Re: Ringeriks-Kraft", "no"),
    ("Re", "Thanks."),
    ("Liliana F. Chiu", "no"),
    ("Re", "beeper - 888-420-6177"),
    ("Expedia", "ksward1;kw2113"),
    ("On-call Procedures/Contacts", "> <<oncall.doc>> - oncall.doc"),
    ("Re: Articles", "Thanks Pearce."),
    ("Re:", "thanks h"),
    ("RE:", "call me at 7138537176"),
    ("RE:", "thanks"),
    ("ARM 2 pager", "here it is"),
    ("Re:", "in or out?"),
    ("nan", "1/1/02 1/2/02 1/7/02 1/9/02 1/11/02 1/13/02 1/15/02 1/16/02 1/21/02"),
    ("", "1/1/02 1/2/02 1/7/02 1/9/02 1/11/02 1/13/02 1/15/02 1/16/02 1/21/02"),
    ("Egyptian Festival", "<<Egyptian Festival.url>> http://www.egyptianfestival.com/ - Egyptian Festival.url"),
    ("nan", "Here at last are the 00's."),
    ("nan", "do I get until 7:00"),

    #("Re", "no"),
]

# Convert the list into a set for efficient lookup
emails_to_remove_set = set(emails_to_remove)

# Filter out matching rows
email_data = email_data[
    ~email_data.apply(lambda row: (row["Subject"], row["Message"]) in emails_to_remove_set, axis=1)
]

print("Specified emails removed successfully!")

Specified emails removed successfully!


In [ ]:
email_data.info()

In [165]:
import re

def clean_subject_line(subject):
    if not isinstance(subject, str):
        return ""

    # 1. Remove common prefixes like "FW:", "Fwd:", "RE:", "[RE]", etc.
    subject = re.sub(r'^\[(FW|FWD|RE|REPLIED|FORWARDED)\]\s*', '', subject)  # Handling brackets around prefixes
    subject = re.sub(r'^(FW:|Fwd:|RE:|Re:|Replied:|Sent:|Reply:|Fwd\[1\]:|Re\[1\]:)\s*', '', subject)

    # 2. Remove excessive spaces
    subject = re.sub(r'\s+', ' ', subject).strip()

    return subject

# Apply to the DataFrame's Subject column
email_data['Cleaned_Subject'] = email_data['Subject'].apply(clean_subject_line)

In [166]:
import re

# Function to remove standalone numbers (like '11') but keep words like 'Warning11'
def remove_standalone_numbers(text):
    text = re.sub(r'\b\d+\b', '', text)  # Remove standalone numbers
    return text.strip()

# Function to remove special characters/symbols (like punctuation)
def remove_special_characters(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove anything that's not alphanumeric or space
    return text

def combine_cleaning_steps(text):
    if not isinstance(text, str):
        # Handle non-string values gracefully
        text = str(text) if text is not None else ''
    
    # First, remove the standalone numbers
    text = remove_standalone_numbers(text)
    
    # Then, remove special characters
    text = remove_special_characters(text)
    
    return text

# Apply the function to the subject column
email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].apply(combine_cleaning_steps)


In [169]:
# Count NaN, None, and empty string values in the 'Cleaned_Subject' column
nan_count = email_data['Cleaned_Subject'].isna().sum() + (email_data['Cleaned_Subject'] == '').sum()

# Print the result
print(f"Number of NaN or empty values in 'Cleaned_Subject': {nan_count}")

Number of NaN or empty values in 'Cleaned_Subject': 16453


In [171]:
#token and stop words
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def tokenize_text(text):
    
    if not text or not isinstance(text, str):
        return []
    
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return filtered_tokens 

email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].apply(tokenize_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [172]:
#lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(tokens):
    if not tokens or not isinstance(tokens, list): 
        return []
    lemmatize_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatize_tokens)

email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].apply(lemmatize_text)
email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [183]:
# Ensure Cleaned_Subject and Cleaned_Message are strings (for safety)
email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].fillna('').astype(str)
email_data['Cleaned_Message'] = email_data['Cleaned_Message'].fillna('').astype(str)

# Combine Subject and Message into Combined_Text
email_data['Combined_Text'] = email_data['Cleaned_Subject'] + " " + email_data['Cleaned_Message']

# Final step: Remove accidental 'nan' (in case subject or message was missing) and extra spaces
email_data['Combined_Text'] = email_data['Combined_Text'].str.replace(r'\b[nN][aA][nN]\b', '', regex=True).str.strip()

C:\Users\User\AppData\Local\Temp\ipykernel_6592\2939605772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_data['Cleaned_Subject'] = email_data['Cleaned_Subject'].fillna('').astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_6592\2939605772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_data['Cleaned_Message'] = email_data['Cleaned_Message'].fillna('').astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_6592\2939605772.py:6: SettingWithCopyWarning: 
A value is trying to be set on

In [184]:
# Count NaN, None, and empty string values in the 'Cleaned_Subject' column
nan_count = email_data['Combined_Text'].isna().sum() + (email_data['Combined_Text'] == '').sum()

# Print the result
print(f"Number of NaN or empty values in 'Combined_Text': {nan_count}")

Number of NaN or empty values in 'Combined_Text': 1


In [185]:
email_data = email_data[email_data['Combined_Text'].notna() & (email_data['Combined_Text'] != '')]

In [186]:
email_data.columns

Index(['From', 'To', 'Subject', 'Message', 'Day', 'Date & Time',
       'Response Time', 'Extracted Metadata', 'url_count',
       'email_address_count', 'attachment_count', 'Cleaned_Message',
       'Cleaned_Subject', 'Combined_Text'],
      dtype='object')

In [187]:
# Rearranging the columns
email_data = email_data[['From', 'To', 'Subject', 'Message', 'Day', 'Date & Time', 
                         'Response Time', 'Extracted Metadata', 'url_count', 
                         'email_address_count', 'attachment_count', 
                         'Cleaned_Subject', 'Cleaned_Message', 'Combined_Text']]

# Check the new order of columns
print(email_data.columns)

Index(['From', 'To', 'Subject', 'Message', 'Day', 'Date & Time',
       'Response Time', 'Extracted Metadata', 'url_count',
       'email_address_count', 'attachment_count', 'Cleaned_Subject',
       'Cleaned_Message', 'Combined_Text'],
      dtype='object')


In [189]:
# Save the DataFrame to a CSV file
email_data.to_csv('cleaned_email_data.csv', index=False)

In [1]:
import pandas as pd
import re

email_data = pd.read_csv('C:/Users/User/OneDrive - Asia Pacific University/APU Final Year Project/FYP Email Project Documents/Email Dataset Python/FYP PART 1/cleaned_email_data.csv')

In [ ]:
email_data = email_data.drop(columns=['From'])
email_data = email_data.drop(columns=['To'])
email_data = email_data.drop(columns=['Day'])
email_data = email_data.drop(columns=['Date & Time'])
email_data = email_data.drop(columns=['Response Time'])
email_data = email_data.drop(columns=['Extracted Metadata'])

In [4]:
email_data

,Subject,Message,url_count,email_address_count,attachment_count,Cleaned_Subject,Cleaned_Message,Combined_Text
0,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,4,0,0,oati etag minimum requirement,time run short company prepare etag minimum re...,oati etag minimum requirement time run short c...
1,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,4,0,0,oati etag minimum requirement,time run short company prepare etag minimum re...,oati etag minimum requirement time run short c...
2,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,4,0,0,oati etag minimum requirement,time run short company prepare etag minimum re...,oati etag minimum requirement time run short c...
3,!!! OATI Etag 1.7 Minimum Requirements !!!,Time is running very short. Is your company pr...,4,0,0,oati etag minimum requirement,time run short company prepare etag minimum re...,oati etag minimum requirement time run short c...
4,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,Immediately delete and DO NOT OPEN email From:...,0,0,0,gonescr virus warning,immediately delete open email alam raf subject...,gonescr virus warning immediately delete open ...
...,...,...,...,...,...,...,...,...
248814,NaN,"both list are on here, the one you are looking...",0,0,0,NaN,list look,list look
248815,NaN,Date Daily Withdrawal Daily Injection Jul-02 3...,0,0,0,NaN,date daily withdrawal injection jul aug sep oc...,date daily withdrawal injection jul aug sep oc...
248816,NaN,good luck. when is comes to daycares. it proba...,0,0,0,NaN,good luck come daycare probably look sense pla...,good luck come daycare probably look sense pla...
248817,NaN,hey sweet cheeks....how the hell are ya????how...,0,0,0,NaN,hey sweet cheek hell ya how bpa bet let know v...,hey sweet cheek hell ya how bpa bet let know v...
